<a href="https://colab.research.google.com/github/LaunchPad-Purdue/Mentor-Mentee-Pairing/blob/main/Pairing_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [197]:
import pandas as pd
import numpy as np

In [198]:
mentees = pd.read_csv("fakeMentees.csv")
mentors = pd.read_csv("fakeMentors.csv")

In [199]:
mentees['Name'] = mentees['Name']
mentees.describe()

,Name
count,45
unique,45
top,Bradley Mcintosh
freq,1


In [200]:
mentors['Name'] = mentors['Name']
mentors.describe()

,Name
count,45
unique,45
top,Isaac Estes
freq,1


## Random generation for testing 

In [201]:
size = 45
matches = 15

In [202]:
import random

def generate_match(max=size):
  val = random.randint(0, max - 1)
  return val

def generate_x_students(max=size):
  arr = []
  while len(arr) < matches:
    num = generate_match(max)
    if num not in arr:
      arr.append(num)
  return arr

In [203]:
# Simulating distribution with 1, 2
test = {0: 0, 1: 0}
for i in range(1, 10000):
  test[generate_match(2)] += 1
print(test)
# Roughly simulating three batches of students

generate_x_students()

{0: 5102, 1: 4897}


[8, 40, 3, 7, 17, 18, 25, 37, 44, 6, 21, 39, 29, 31, 12]

In [204]:
# Randomizer for simulating responses

mentee_ranks = pd.DataFrame() # What mentors ranked mentees
mentor_ranks = pd.DataFrame() # What mentees ranked mentors

for i in mentors['Name']: # Create a bunch of random randomized 1-10 rankings for mentors
  df = {"Timestamp": "10",
         "Email address": "%s@purdue.edu" % i.lower(),
         "What's your name?": i}
  arr = generate_x_students()
  df["Rank 1 (Top Choice)"] = mentees["Name"][arr[0]]
  for j in range(1, matches):
    df["Rank %d" % (j + 1)] = mentees["Name"][arr[j]]
  mentee_ranks = mentee_ranks.append(df, ignore_index=True)

for i in mentees['Name']:
  df = {"Timestamp": "10",
         "Email address": "%s@purdue.edu" % i.lower(),
         "What's your name?": i}
  arr = generate_x_students()
  df["Rank 1 (Top Choice)"] = mentors["Name"][arr[0]]
  for j in range(1, matches):
    df["Rank %d" % (j + 1)] = mentors["Name"][arr[j]]
  mentor_ranks = mentor_ranks.append(df, ignore_index=True)

# This is all basically to get a replica of the google form responses that'll be loaded in

In [205]:
mentee_ranks.head()

,Timestamp,Email address,What's your name?,Rank 1 (Top Choice),Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10,Rank 11,Rank 12,Rank 13,Rank 14,Rank 15
0,10,isaac estes@purdue.edu,Isaac Estes,Diego Hargreeves,Adam Young,Rowena Fitzpatrick,Cristobal Gordon,Everett Aguilar,Rosalind Mata,Dean George,Luther Hargreeves,Ben Hargreeves,Walker Murillo,John Bonilla,Rhoda Cummings,Taylor Swift,Billy Munoz,Victor Hargreeves
1,10,aileen lin@purdue.edu,Aileen Lin,Brianna Lara,Dean George,Lana Delrey,Lawanda Allison,Franklin Hayden,Billy Munoz,Dr Manhattan,Trenton Woods,Kasey Hickman,Victor Hargreeves,Rhoda Cummings,John Bonilla,Hubert Nguyen,Becky Ashley,Allison Hargreeves
2,10,jan fowler@purdue.edu,Jan Fowler,Ben Hargreeves,John Bonilla,Kris Fisher,Adam Young,Diego Hargreeves,Mark Grayson,Brianna Lara,Rosalind Mata,Althea Stevens,Rowena Fitzpatrick,Stefan Stewart,Cristobal Gordon,Deangelo Fuller,Luther Hargreeves,Erica Bradley
3,10,donnie boone@purdue.edu,Donnie Boone,Ben Hargreeves,Becky Ashley,Shahrukh Khan,Franklin Hayden,Klaus Hargreeves,Rowena Fitzpatrick,John Constantine,Lana Delrey,Sean Rodriguez,Bradley Mcintosh,Billy Butcher,Irene Melton,Paula Torres,Rhoda Cummings,Vanya Hargreeves
4,10,sammy brooks@purdue.edu,Sammy Brooks,Sean Rodriguez,Simone Doyle,Brianna Lara,Franklin Hayden,Deangelo Fuller,Dean George,Taylor Swift,Stefan Stewart,Everett Aguilar,Allison Hargreeves,Kasey Hickman,Billy Munoz,Victor Hargreeves,Irene Melton,Luther Hargreeves


In [206]:
mentor_ranks.head()

,Timestamp,Email address,What's your name?,Rank 1 (Top Choice),Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10,Rank 11,Rank 12,Rank 13,Rank 14,Rank 15
0,10,bradley mcintosh@purdue.edu,Bradley Mcintosh,Genaro Dunlap,Linda Anderson,Lynette Mccarthy,Joe Frazier,Sammy Brooks,Eleanor Newton,Louis Wilson,Abdul Byrd,Mike Anderson,Bill Frazier,Ronnie Mccarthy,Donnie Boone,Yvette Mccormick,Gilbert Rosario,Hung Cunningham
1,10,trenton woods@purdue.edu,Trenton Woods,James Francis,Joey Anderson,Sammy Brooks,Kathleen Donaldson,Teddy Hogan,Eleanor Newton,Donte Li,Hiram Moreno,Ronnie Mccarthy,Becky Mccarthy,Isaac Estes,Arturo Huber,Darcy Daniel,Kaye Chang,Yvette Mccormick
2,10,brianna lara@purdue.edu,Brianna Lara,Joey Anderson,Carmela Malone,Donte Li,Kent Villarreal,Erma Mcclain,Genaro Dunlap,Lynette Mccarthy,Teddy Hogan,Damian Hood,Travis Lloyd,Yvette Mccormick,Elijah Mccall,Latonya Knapp,Gilbert Rosario,Sammy Brooks
3,10,cristobal gordon@purdue.edu,Cristobal Gordon,Travis Lloyd,Eleanor Newton,Ronnie Mccarthy,Paul Mccarthy,Brandy Branch,Teddy Hogan,Yvette Mccormick,Quinn Brewer,Louis Wilson,Aileen Lin,Jan Fowler,Abdul Byrd,Joe Frazier,Hiram Moreno,Berta Lindsey
4,10,billy munoz@purdue.edu,Billy Munoz,Linda Cline,Kathleen Donaldson,Kent Villarreal,Darcy Daniel,Joey Anderson,Paul Mccarthy,Louis Wilson,Sammy Brooks,Yvette Mccormick,Donte Li,Donnie Boone,Peter Carry,Saul Mccarthy,Quinn Brewer,Erma Mcclain


# Actual Algorithm
Based on: https://rosettacode.org/wiki/Stable_marriage_problem#Python


In [207]:
# Now here's where the actual algorithm starts.
# Insert stuff to read csv files for mentor mentee here
## mentor_ranks = pd.read_csv("menteesresp.csv")
## mentee_ranks = pd.read_csv("mentorsresp.csv")

In [208]:
mentor_ranks.head()

,Timestamp,Email address,What's your name?,Rank 1 (Top Choice),Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10,Rank 11,Rank 12,Rank 13,Rank 14,Rank 15
0,10,bradley mcintosh@purdue.edu,Bradley Mcintosh,Genaro Dunlap,Linda Anderson,Lynette Mccarthy,Joe Frazier,Sammy Brooks,Eleanor Newton,Louis Wilson,Abdul Byrd,Mike Anderson,Bill Frazier,Ronnie Mccarthy,Donnie Boone,Yvette Mccormick,Gilbert Rosario,Hung Cunningham
1,10,trenton woods@purdue.edu,Trenton Woods,James Francis,Joey Anderson,Sammy Brooks,Kathleen Donaldson,Teddy Hogan,Eleanor Newton,Donte Li,Hiram Moreno,Ronnie Mccarthy,Becky Mccarthy,Isaac Estes,Arturo Huber,Darcy Daniel,Kaye Chang,Yvette Mccormick
2,10,brianna lara@purdue.edu,Brianna Lara,Joey Anderson,Carmela Malone,Donte Li,Kent Villarreal,Erma Mcclain,Genaro Dunlap,Lynette Mccarthy,Teddy Hogan,Damian Hood,Travis Lloyd,Yvette Mccormick,Elijah Mccall,Latonya Knapp,Gilbert Rosario,Sammy Brooks
3,10,cristobal gordon@purdue.edu,Cristobal Gordon,Travis Lloyd,Eleanor Newton,Ronnie Mccarthy,Paul Mccarthy,Brandy Branch,Teddy Hogan,Yvette Mccormick,Quinn Brewer,Louis Wilson,Aileen Lin,Jan Fowler,Abdul Byrd,Joe Frazier,Hiram Moreno,Berta Lindsey
4,10,billy munoz@purdue.edu,Billy Munoz,Linda Cline,Kathleen Donaldson,Kent Villarreal,Darcy Daniel,Joey Anderson,Paul Mccarthy,Louis Wilson,Sammy Brooks,Yvette Mccormick,Donte Li,Donnie Boone,Peter Carry,Saul Mccarthy,Quinn Brewer,Erma Mcclain


## Preprocessing

Since Stable Marriage requires an (2N)x(N) mapping, we're randomly ranking people between 11-30 to get to a 60 x 30 mapping.

Our assumption is that most matchings would happen between people who ranked each other in their top 10. We'll then figure out people who didn't get matches between 1-10 and manually review their profiles and match them.

In [209]:
tor_p = {} #mentor preferences
tee_p = {} #mentee preferences

tors = mentors.to_numpy().reshape(size)
tees = mentees.to_numpy().reshape(size)

# Setup dictionaries with names as keys

for i in tors:
  tor_p[i] = [] 

for i in tees:
  tee_p[i] = []

# Building mentor ranks
for i in mentee_ranks.iterrows():
  name = i[1]["What's your name?"]
  tor_p[name].append(i[1]["Rank 1 (Top Choice)"])
  for j in range(1, matches):
    tor_p[name].append(i[1]["Rank %d" % (j + 1)]) # Add 1-10 rankings to each person
  remaining = np.setdiff1d(tees, tor_p[name])
  np.random.shuffle(remaining) # Shuffle remaining 11-20 rankings (see explanation above)
  for j in remaining:
    tor_p[name].append(j)
print("Mentor Rankings: ", len(tor_p), tor_p)

# Building mentee ranks
for i in mentor_ranks.iterrows():
  name = i[1]["What's your name?"]
  tee_p[name].append(i[1]["Rank 1 (Top Choice)"])
  for j in range(1, matches):
    tee_p[name].append(i[1]["Rank %d" % (j + 1)])
  remaining = np.setdiff1d(tors, tee_p[name])
  np.random.shuffle(remaining)
  for j in remaining:
    tee_p[name].append(j)
print("Mentee Rankings: ", len(tee_p), tee_p)

Mentor Rankings:  45 {'Isaac Estes': ['Diego Hargreeves', 'Adam Young', 'Rowena Fitzpatrick', 'Cristobal Gordon', 'Everett Aguilar', 'Rosalind Mata', 'Dean George', 'Luther Hargreeves', 'Ben Hargreeves', 'Walker Murillo', 'John Bonilla', 'Rhoda Cummings', 'Taylor Swift', 'Billy Munoz', 'Victor Hargreeves', 'Allison Hargreeves', 'Mark Grayson', 'Klaus Hargreeves', 'Sean Rodriguez', 'Erica Bradley', 'Althea Stevens', 'Franklin Hayden', 'Vanya Hargreeves', 'Shahrukh Khan', 'Deangelo Fuller', 'Teddy Whitehead', 'Billy Butcher', 'Dianna Gay', 'Paula Torres', 'John Constantine', 'Kris Fisher', 'Kasey Hickman', 'Catherine Mosley', 'Dr Manhattan', 'Trenton Woods', 'Lana Delrey', 'Brianna Lara', 'Irene Melton', 'Simone Doyle', 'Hobert Carr', 'Stefan Stewart', 'Hubert Nguyen', 'Becky Ashley', 'Bradley Mcintosh', 'Lawanda Allison'], 'Aileen Lin': ['Brianna Lara', 'Dean George', 'Lana Delrey', 'Lawanda Allison', 'Franklin Hayden', 'Billy Munoz', 'Dr Manhattan', 'Trenton Woods', 'Kasey Hickman', 'V

## Stable Marriage

This does a better explanation than I could: https://en.wikipedia.org/wiki/Stable_marriage_problem

In [210]:
import copy

## Disclosure: This is copied from https://rosettacode.org/wiki/Stable_marriage_problem#Python

guyprefers = tor_p
galprefers = tee_p
guys = sorted(tor_p.keys())
gals = sorted(tee_p.keys()) 

def check(engaged):
    inverseengaged = dict((v,k) for k,v in engaged.items())
    for she, he in engaged.items():
        shelikes = galprefers[she]
        shelikesbetter = shelikes[:shelikes.index(he)]
        helikes = guyprefers[he]
        helikesbetter = helikes[:helikes.index(she)]
        for guy in shelikesbetter:
            guysgirl = inverseengaged[guy]
            guylikes = guyprefers[guy]
            if guylikes.index(guysgirl) > guylikes.index(she):
                print("%s and %s like each other better than "
                      "their present partners: %s and %s, respectively"
                      % (she, guy, he, guysgirl))
                return False
        for gal in helikesbetter:
            girlsguy = engaged[gal]
            gallikes = galprefers[gal]
            if gallikes.index(girlsguy) > gallikes.index(he):
                print("%s and %s like each other better than "
                      "their present partners: %s and %s, respectively"
                      % (he, gal, she, girlsguy))
                return False
    return True
 
def matchmaker():
    guysfree = guys[:]
    engaged  = {}
    guyprefers2 = copy.deepcopy(guyprefers)
    galprefers2 = copy.deepcopy(galprefers)
    while guysfree:
        guy = guysfree.pop(0)
        guyslist = guyprefers2[guy]
        gal = guyslist.pop(0)
        fiance = engaged.get(gal)
        if not fiance:
            # She's free
            engaged[gal] = guy
            print("  %s and %s" % (guy, gal))
        else:
            # The bounder proposes to an engaged lass!
            galslist = galprefers2[gal]
            if galslist.index(fiance) > galslist.index(guy):
                # She prefers new guy
                engaged[gal] = guy
                print("  %s dumped %s for %s" % (gal, fiance, guy))
                if guyprefers2[fiance]:
                    # Ex has more girls to try
                    guysfree.append(fiance)
            else:
                # She is faithful to old fiance
                if guyslist:
                    # Look again
                    guysfree.append(guy)
    return engaged
 
 
print('\nPairings:')
engaged = matchmaker()
 
print('\nMentor-Mentee Pairs:')
print('  ' + ',\n  '.join('%s is paired to %s' % couple
                          for couple in sorted(engaged.items())))
print()
print('Pairing stability check PASSED'
      if check(engaged) else 'Pairing stability check FAILED')


Pairings:
  Abdul Byrd and Teddy Whitehead
  Aileen Lin and Brianna Lara
  Brianna Lara dumped Aileen Lin for Arturo Huber
  Becky Mccarthy and Rhoda Cummings
  Berta Lindsey and Vanya Hargreeves
  Bill Frazier and Klaus Hargreeves
  Brandy Branch and Kris Fisher
  Carmela Malone and Dianna Gay
  Damian Hood and Stefan Stewart
  Darcy Daniel and Dr Manhattan
  Donnie Boone and Ben Hargreeves
  Donte Li and Sean Rodriguez
  Eleanor Newton and Dean George
  Elijah Mccall and Luther Hargreeves
  Erma Mcclain and Billy Munoz
  Genaro Dunlap and Deangelo Fuller
  Gilbert Rosario and Victor Hargreeves
  Hiram Moreno and Allison Hargreeves
  Homer Irwin and Taylor Swift
  Hung Cunningham and Kasey Hickman
  Isaac Estes and Diego Hargreeves
  James Francis and Everett Aguilar
  Kasey Hickman dumped Hung Cunningham for Joe Frazier
  Joey Anderson and Franklin Hayden
  Billy Munoz dumped Erma Mcclain for Kathleen Donaldson
  Kaye Chang and Becky Ashley
  Brianna Lara dumped Arturo Huber for Ken

In [211]:
import json
print(json.dumps(engaged,sort_keys=True, indent=4))
print(len(engaged))

{
    "Adam Young": "Linda Cline",
    "Allison Hargreeves": "Genaro Dunlap",
    "Althea Stevens": "Donte Li",
    "Becky Ashley": "Damian Hood",
    "Ben Hargreeves": "Louis Wilson",
    "Billy Butcher": "Berta Lindsey",
    "Billy Munoz": "Kathleen Donaldson",
    "Bradley Mcintosh": "Yvette Mccormick",
    "Brianna Lara": "Carmela Malone",
    "Catherine Mosley": "Linda Anderson",
    "Cristobal Gordon": "Travis Lloyd",
    "Dean George": "Peter Carry",
    "Deangelo Fuller": "Sammy Brooks",
    "Dianna Gay": "Erma Mcclain",
    "Diego Hargreeves": "Isaac Estes",
    "Dr Manhattan": "Kaye Chang",
    "Erica Bradley": "Latonya Knapp",
    "Everett Aguilar": "Mike Anderson",
    "Franklin Hayden": "Donnie Boone",
    "Hobert Carr": "Teddy Hogan",
    "Hubert Nguyen": "Aileen Lin",
    "Irene Melton": "Paul Mccarthy",
    "John Bonilla": "Darcy Daniel",
    "John Constantine": "Hiram Moreno",
    "Kasey Hickman": "Joe Frazier",
    "Klaus Hargreeves": "Hung Cunningham",
    "Kris Fish

In [212]:
# Figuring out true matches (Where both mentors and mentees actually ranked the people)
results = pd.DataFrame()
true_matches = {}
for k in engaged:
  v = engaged[k]
  if tor_p[v].index(k) < matches and tee_p[k].index(v) < matches: # Either one has been selected using random picks
    true_matches[k + ", " + v] = str(tee_p[k].index(v)) + ", " + str(tor_p[v].index(k))
    df = {}
    df["Mentor Name"] = v
    df["Mentee Name"] = k
    df["Mentee's Rating"] = tee_p[k].index(v) + 1
    df["Mentor's Rating"] = tor_p[v].index(k) + 1
    results = results.append(df, ignore_index=True)
print(json.dumps(true_matches,sort_keys=True, indent=4))
print(len(true_matches))

{
    "Adam Young, Linda Cline": "3, 6",
    "Althea Stevens, Donte Li": "9, 1",
    "Becky Ashley, Damian Hood": "5, 6",
    "Ben Hargreeves, Louis Wilson": "1, 11",
    "Billy Butcher, Berta Lindsey": "2, 4",
    "Billy Munoz, Kathleen Donaldson": "1, 0",
    "Bradley Mcintosh, Yvette Mccormick": "12, 2",
    "Brianna Lara, Carmela Malone": "1, 1",
    "Catherine Mosley, Linda Anderson": "11, 0",
    "Cristobal Gordon, Travis Lloyd": "0, 0",
    "Dean George, Peter Carry": "5, 4",
    "Deangelo Fuller, Sammy Brooks": "6, 4",
    "Diego Hargreeves, Isaac Estes": "2, 0",
    "Dr Manhattan, Kaye Chang": "5, 3",
    "Franklin Hayden, Donnie Boone": "0, 3",
    "Hobert Carr, Teddy Hogan": "13, 6",
    "John Bonilla, Darcy Daniel": "6, 3",
    "Kasey Hickman, Joe Frazier": "0, 0",
    "Klaus Hargreeves, Hung Cunningham": "1, 1",
    "Lana Delrey, Kent Villarreal": "7, 2",
    "Lawanda Allison, Eleanor Newton": "7, 1",
    "Luther Hargreeves, Elijah Mccall": "2, 0",
    "Mark Grayson, Homer

In [213]:
# Figuring out matches made by randomization:

random_found = {}
for k in engaged:
  v = engaged[k]
  if tor_p[v].index(k) >= matches or tee_p[k].index(v) >= matches: # Either one has been selected using random picks
    random_found[k + ", " + v] = str(tee_p[k].index(v)) + ", " + str(tor_p[v].index(k))
    df = {}
    df["Mentor Name"] = v
    df["Mentee Name"] = k
    df["Mentee's Rating"] = tee_p[k].index(v) + 1
    df["Mentor's Rating"] = tor_p[v].index(k) + 1
    results = results.append(df, ignore_index=True)
print(json.dumps(random_found,sort_keys=True, indent=4))
print(len(random_found))

{
    "Allison Hargreeves, Genaro Dunlap": "21, 7",
    "Dianna Gay, Erma Mcclain": "23, 4",
    "Erica Bradley, Latonya Knapp": "34, 3",
    "Everett Aguilar, Mike Anderson": "10, 21",
    "Hubert Nguyen, Aileen Lin": "44, 12",
    "Irene Melton, Paul Mccarthy": "38, 2",
    "John Constantine, Hiram Moreno": "1, 16",
    "Kris Fisher, Jan Fowler": "35, 2",
    "Paula Torres, Paul Cline": "23, 7",
    "Vanya Hargreeves, Becky Mccarthy": "24, 7"
}
10


The randomized dataset had estimated around 10/30 pairs wouldn't get matches. But we didn't consider that if a mentee ranks a mentor high, the mentor is likely to rank the mentee high. And vice-versa. Both of these considerations would make it more likely that people rank each other in their top 10.

We didn't really get into conditional probabilities in this simulation, but if we did consider those we'd have even fewer pairs that needed to be matched manually.

In [214]:
results = results.reindex(columns=["Mentee Name", "Mentor Name", "Mentee's Rating", "Mentor's Rating"])
results["Mentee's Rating"] = pd.to_numeric(results["Mentee's Rating"], downcast="integer")
results["Mentor's Rating"] = pd.to_numeric(results["Mentor's Rating"], downcast="integer")
results.head()

,Mentee Name,Mentor Name,Mentee's Rating,Mentor's Rating
0,Teddy Whitehead,Abdul Byrd,5,1
1,Brianna Lara,Carmela Malone,2,2
2,Rhoda Cummings,Saul Mccarthy,2,1
3,Klaus Hargreeves,Hung Cunningham,2,2
4,Stefan Stewart,Brandy Branch,6,7


In [215]:
results.to_csv("pairs.csv", index=False)